In [ ]:
#GPR weight space
import numpy as np
from scipy.linalg import cho_factor, cho_solve

class GPRegressor:
    def __init__(self, kernel):
        self.kernel = kernel
        self.X = None
        self.y = None
        self.alpha = None

    def fit(self, X, y):
        self.X = X
        self.y = y

        # Compute the kernel matrix
        K = self.kernel(X, X)

        # Add noise to the diagonal for numerical stability
        noise = 1e-6 * np.eye(len(X))
        K += noise

        # Compute the Cholesky factorization of the kernel matrix
        L, lower = cho_factor(K, lower=True)

        # Solve for alpha = K^-1 y
        self.alpha = cho_solve((L, lower), y)

    def predict(self, X_test):
        # Compute the kernel matrix between test inputs and training inputs
        K_test = self.kernel(X_test, self.X)

        # Compute the mean of the predictive distribution
        y_mean = np.dot(K_test, self.alpha)

        # Compute the covariance of the predictive distribution
        K_inv = cho_solve((L, lower), np.eye(len(self.X)))
        y_cov = self.kernel(X_test, X_test) - np.dot(K_test, np.dot(K_inv, K_test.T))

        return y_mean, y_cov

In [ ]:
#GPR function space

import numpy as np

class GPRegressor:
    def __init__(self, kernel):
        self.kernel = kernel
        self.X = None
        self.y = None
        self.K = None

    def fit(self, X, y):
        self.X = X
        self.y = y

        # Compute the kernel matrix
        self.K = self.kernel(X, X)

        # Add noise to the diagonal for numerical stability
        noise = 1e-6 * np.eye(len(X))
        self.K += noise

    def predict(self, X_test):
        # Compute the kernel matrix between test inputs and training inputs
        K_test = self.kernel(X_test, self.X)

        # Compute the mean of the predictive distribution
        f_mean = np.dot(K_test, np.dot(np.linalg.inv(self.K), self.y))

        # Compute the covariance of the predictive distribution
        f_cov = self.kernel(X_test, X_test) - np.dot(K_test, np.dot(np.linalg.inv(self.K), K_test.T))

        return f_mean, f_cov

In [20]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import GPy

# Generate random input data
X = np.random.rand(1000, 100)

# Generate corresponding output data
y = np.sin(X[:, 0] * 10) + np.random.randn(1000) * 0.1

# Define kernel function for scikit-learn
kernel_sk = RBF(length_scale=1.0)

# Define kernel function for GPy
kernel_gpy = GPy.kern.RBF(input_dim=100, variance=1.0, lengthscale=1.0)

# Create scikit-learn model
model_sk = GaussianProcessRegressor(kernel=kernel_sk)

# Create GPy model
model_gpy = GPy.models.GPRegression(X, np.expand_dims(y, 1), kernel_gpy)

# Train scikit-learn model
model_sk.fit(X, y)

# Train GPy model
model_gpy.optimize()

# Generate new test data
X_test = np.random.rand(20, 100)
y = np.sin(X_test[:, 0] * 100)

# Predict using scikit-learn model
y_pred_sk, y_std_sk = model_sk.predict(X_test, return_std=True)

# Predict using GPy model
y_pred_gpy, y_var_gpy = model_gpy.predict(X_test)

# Print predictions
print("scikit-learn predictions:\n", np.mean(abs(y_pred_sk-y)))
print("GPy predictions:\n", np.mean(abs(y_pred_gpy-y)))

scikit-learn predictions:
 0.6367273762385068
GPy predictions:
 0.6656172161642491


In [21]:
import os
import re
import qml
import numpy as np
import matplotlib.pyplot as plt 
import random
from qml.kernels import gaussian_kernel
from sklearn.kernel_ridge import KernelRidge

import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist

import torch
from pyro.infer import NUTS
from pyro.infer.mcmc import MCMC
from pyro.infer.mcmc.hmc import HMC

from torch.nn import Parameter



from pyro.contrib.autoguide import AutoMultivariateNormal
from sklearn.utils import shuffle
from pyro.infer.util import torch_backward, torch_item

from time import time
from time import process_time
random.seed(123)

In [23]:
X = np.load("./data/X.npy")
Y = np.loadtxt("./data/E_def2-tzvp.dat")
X_test = X[-1000:]
Y_test = Y[-1000:]

In [24]:
kernel = gp.kernels.Exponential(input_dim = 2, variance=torch.tensor(1/0.004641588833612777))

In [25]:
X_test_tensor = torch.from_numpy(X_test)
Y_test_tensor = torch.from_numpy(Y_test)

number = 9000
X_tensor = torch.from_numpy(X[:number])
Y_tensor = torch.from_numpy(Y[:number])

In [26]:
X_new = Parameter(X_tensor.clone())
x, y = shuffle(X_new, Y_tensor)

gpr = gp.models.GPRegression(X_new, Y_tensor, kernel)
optimizer = torch.optim.Adam(gpr.parameters(), lr=0.005)
autoGuide = AutoMultivariateNormal(gpr)
loss_fn = pyro.infer.TraceMeanField_ELBO().differentiable_loss
losses = []
locations = []
variances = []
lengthscales = []
noises = []
iterations = 100
regular_time = []
cpu_time = []
t0 = time()
t0_cpu = process_time()
mae = []


for i in range(iterations):
    optimizer.zero_grad()
    variances.append(gpr.kernel.variance.item())
    noises.append(gpr.noise.item())
    lengthscales.append(gpr.kernel.lengthscale.item())
    loss = loss_fn(gpr.model, gpr.guide)
    pro = process_time()
    tim = time()
    torch_backward(loss, True)
    optimizer.step()
    regular_time.append(time()-tim)
    cpu_time.append(process_time()-pro)
    losses.append(loss.item())
#     f, _ = gpr.forward(X_test_tensor)
#     a = torch.abs(Y_test_tensor - f)
#     mae.append(torch.sum(a) / 1000)

# loss = gp.util.train(gpr, optimizer=optimizer, num_steps=iterations)
losses = list(map(lambda x: x/number, losses))


KeyboardInterrupt: 